# US Healthcare Project (DE SynPUF)

## Catalog Configuration

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS us_healthcare_synpuf;

In [0]:
%sql
USE CATALOG us_healthcare_synpuf;

## Bronze Layer Configuration

In [0]:
%sql
create schema if not exists bronze;

python code to unzip the CMS DE-SynPUF files into csv

created "raw_vol" volume using UI

In [0]:
import os
import zipfile
import shutil

# the source and destination paths
zip_dir = "/Volumes/us_healthcare_synpuf/bronze/raw_vol/zip/"
csv_dir = "/Volumes/us_healthcare_synpuf/bronze/raw_vol/csv/"
temp_extract_dir = "/tmp/healthcare_extraction/"

os.makedirs(csv_dir, exist_ok=True)
if os.path.exists(temp_extract_dir):
    shutil.rmtree(temp_extract_dir)
os.makedirs(temp_extract_dir)

# 1. Loop through all zip files in the source directory
for file_name in os.listdir(zip_dir):
    if file_name.endswith(".zip"):
        zip_path = os.path.join(zip_dir, file_name)
        
        print(f"Processing: {file_name}")
        
        # 2. Extract the zip to a temporary location
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(temp_extract_dir)
        
        # 3. Walk through the extracted files to find CSVs
        # This handles cases where the zip contains folders
        for root, dirs, files in os.walk(temp_extract_dir):
            for file in files:
                if file.lower().endswith(".csv"):
                    source_file = os.path.join(root, file)
                    destination_file = os.path.join(csv_dir, file)
                    
                    shutil.move(source_file, destination_file)
                    print(f"  -> Extracted and moved: {file}")

        # 4. Clean up the temp folder for the next zip file
        for item in os.listdir(temp_extract_dir):
            item_path = os.path.join(temp_extract_dir, item)
            if os.path.isdir(item_path):
                shutil.rmtree(item_path)
            else:
                os.remove(item_path)

print("\nAll CSVs have been successfully extracted and flattened into the CSV volume!")

Processing: 176541_DE1_0_2008_Beneficiary_Summary_File_Sample_1.zip
  -> Extracted and moved: DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv
Processing: 176549_DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.zip
  -> Extracted and moved: DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv
Processing: 176600_DE1_0_2009_Beneficiary_Summary_File_Sample_1.zip
  -> Extracted and moved: DE1_0_2009_Beneficiary_Summary_File_Sample_1.csv
Processing: 176616_DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.zip
  -> Extracted and moved: DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv
Processing: DE1_0_2008_to_2010_Carrier_Claims_Sample_1A.zip
  -> Extracted and moved: DE1_0_2008_to_2010_Carrier_Claims_Sample_1A.csv
Processing: DE1_0_2008_to_2010_Carrier_Claims_Sample_1B.zip
  -> Extracted and moved: DE1_0_2008_to_2010_Carrier_Claims_Sample_1B.csv
Processing: DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_1.zip
  -> Extracted and moved: DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_1.csv
Proces

In [0]:
%sql
drop table if exists bronze.ben_sum;
create table bronze.ben_sum();

In [0]:
%sql
COPY INTO bronze.ben_sum
FROM (
  SELECT 
    *, 
    2008 AS BEN_INSERTED_YEAR 
  FROM '/Volumes/us_healthcare_synpuf/bronze/raw_vol/csv/DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
116352,116352,0


In [0]:
%sql
copy into bronze.ben_sum
FROM (
  SELECT 
    *, 
    2009 AS BEN_INSERTED_YEAR 
  FROM '/Volumes/us_healthcare_synpuf/bronze/raw_vol/csv/DE1_0_2009_Beneficiary_Summary_File_Sample_1.csv'
)
fileformat = CSV
format_options ('header' = 'true', 'inferSchema' = 'true')
copy_options ('mergeSchema' = 'true')

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
114538,114538,0


In [0]:
%sql
copy into bronze.ben_sum
FROM (
  SELECT 
    *, 
    2010 AS BEN_INSERTED_YEAR 
  FROM '/Volumes/us_healthcare_synpuf/bronze/raw_vol/csv/DE1_0_2010_Beneficiary_Summary_File_Sample_1.csv'
)
fileformat = CSV
format_options ('header' = 'true', 'inferSchema' = 'true')
copy_options ('mergeSchema' = 'true')

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
112754,112754,0


In [0]:
%sql
ALTER TABLE bronze.ben_sum SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');
ALTER TABLE bronze.ben_sum RENAME COLUMN BEN_INSERTED_YEAR TO BENE_INSERTED_YEAR;

In [0]:
%sql
select * from information_schema.columns

table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,full_data_type,data_type,character_maximum_length,character_octet_length,numeric_precision,numeric_precision_radix,numeric_scale,datetime_precision,interval_type,interval_precision,maximum_cardinality,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_system_time_period_start,is_system_time_period_end,system_time_period_timestamp_generation,is_updatable,partition_index,comment
us_healthcare_synpuf,bronze,ben_sum,DESYNPUF_ID,0,null,YES,string,STRING,0,0,null,null,null,null,null,null,null,NO,null,null,null,null,null,null,NO,null,NO,NO,null,YES,null,null
us_healthcare_synpuf,bronze,ben_sum,BENE_BIRTH_DT,1,null,YES,int,INT,null,null,null,null,null,null,null,null,null,NO,null,null,null,null,null,null,NO,null,NO,NO,null,YES,null,null
us_healthcare_synpuf,bronze,ben_sum,BENE_DEATH_DT,2,null,YES,int,INT,null,null,null,null,null,null,null,null,null,NO,null,null,null,null,null,null,NO,null,NO,NO,null,YES,null,null
us_healthcare_synpuf,bronze,ben_sum,BENE_SEX_IDENT_CD,3,null,YES,int,INT,null,null,null,null,null,null,null,null,null,NO,null,null,null,null,null,null,NO,null,NO,NO,null,YES,null,null
us_healthcare_synpuf,bronze,ben_sum,BENE_RACE_CD,4,null,YES,int,INT,null,null,null,null,null,null,null,null,null,NO,null,null,null,null,null,null,NO,null,NO,NO,null,YES,null,null
us_healthcare_synpuf,bronze,ben_sum,BENE_ESRD_IND,5,null,YES,string,STRING,0,0,null,null,null,null,null,null,null,NO,null,null,null,null,null,null,NO,null,NO,NO,null,YES,null,null
us_healthcare_synpuf,bronze,ben_sum,SP_STATE_CODE,6,null,YES,int,INT,null,null,null,null,null,null,null,null,null,NO,null,null,null,null,null,null,NO,null,NO,NO,null,YES,null,null
us_healthcare_synpuf,bronze,ben_sum,BENE_COUNTY_CD,7,null,YES,int,INT,null,null,null,null,null,null,null,null,null,NO,null,null,null,null,null,null,NO,null,NO,NO,null,YES,null,null
us_healthcare_synpuf,bronze,ben_sum,BENE_HI_CVRAGE_TOT_MONS,8,null,YES,int,INT,null,null,null,null,null,null,null,null,null,NO,null,null,null,null,null,null,NO,null,NO,NO,null,YES,null,null
us_healthcare_synpuf,bronze,ben_sum,BENE_SMI_CVRAGE_TOT_MONS,9,null,YES,int,INT,null,null,null,null,null,null,null,null,null,NO,null,null,null,null,null,null,NO,null,NO,NO,null,YES,null,null


In [0]:
%sql
drop table if exists bronze.inp_claims;
create table bronze.inp_claims()

In [0]:
%sql
copy into bronze.inp_claims
FROM '/Volumes/us_healthcare_synpuf/bronze/raw_vol/csv/DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv'
fileformat = CSV
format_options ('header' = 'true', 'inferSchema' = 'true')
copy_options ('mergeSchema' = 'true')

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
66773,66773,0


In [0]:
%sql
create table bronze.out_claims()

In [0]:
%sql
    
copy into bronze.out_claims
from '/Volumes/us_healthcare_synpuf/bronze/raw_vol/csv/DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv'
fileformat = CSV
format_options ('header' = 'true', 'inferSchema' = 'true')
copy_options ('mergeSchema' = 'true')

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
790790,790790,0


In [0]:
%sql
create table bronze.pde_claims

In [0]:
%sql
copy into bronze.pde_claims
from '/Volumes/us_healthcare_synpuf/bronze/raw_vol/csv/DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_1.csv'
fileformat = CSV
format_options ('header' = 'true', 'inferSchema' = 'true')
copy_options ('mergeSchema' = 'true')

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
5552421,5552421,0


In [0]:
%sql
SHOW TABLES IN bronze

database,tableName,isTemporary
bronze,ben_sum,false
bronze,car_claims,false
bronze,inp_claims,false
bronze,out_claims,false
bronze,pde_claims,false


In [0]:
%sql
create table bronze.car_claims()

In [0]:
%sql
copy into bronze.car_claims
from '/Volumes/us_healthcare_synpuf/bronze/raw_vol/csv/DE1_0_2008_to_2010_Carrier_Claims_Sample_1A.csv'
fileformat = CSV
format_options ('header' = 'true', 'inferSchema' = 'true')
copy_options ('mergeSchema' = 'true')

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
2370667,2370667,0


In [0]:
%sql
copy into bronze.car_claims
from '/Volumes/us_healthcare_synpuf/bronze/raw_vol/csv/DE1_0_2008_to_2010_Carrier_Claims_Sample_1B.csv'
fileformat = CSV
format_options ('header' = 'true', 'inferSchema' = 'true')
copy_options ('mergeSchema' = 'true')

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
2370668,2370668,0


In [0]:
%sql
select count(*) from bronze.car_claims

count(*)
4741335


In [0]:
%sql
SHOW TABLES IN bronze

database,tableName,isTemporary
bronze,ben_sum,false
bronze,car_claims,false
bronze,inp_claims,false
bronze,out_claims,false
bronze,pde_claims,false


## Silver Layer Configuration

In [0]:
%sql
create schema if not exists silver

In [0]:
%sql
-- beneficiary summary

CREATE OR REPLACE TABLE silver.ben_sum AS
SELECT * FROM (
  SELECT 
    sha2(concat(DESYNPUF_ID, 'HEALTHCARE_SALT_2026'), 256) AS patient_id_hashed,
    CASE WHEN BENE_SEX_IDENT_CD = 1 THEN 'Male' ELSE 'Female' END AS gender,
    SP_STATE_CODE AS state_code,
    -- Chronic Condition Flags (1 = Yes, 2 = No in raw data; converting to 1/0)
    CASE WHEN SP_ALZHDMTA = 1 THEN 1 ELSE 0 END AS flag_alzheimer,
    CASE WHEN SP_CHF = 1 THEN 1 ELSE 0 END AS flag_heart_failure,
    CASE WHEN SP_CHRNKIDN = 1 THEN 1 ELSE 0 END AS flag_kidney_disease,
    CASE WHEN SP_DIABETES = 1 THEN 1 ELSE 0 END AS flag_diabetes,
    BENE_INSERTED_YEAR,
    row_number() OVER (PARTITION BY DESYNPUF_ID ORDER BY BENE_INSERTED_YEAR DESC) as latest_rank
  FROM bronze.ben_sum
)
WHERE latest_rank = 1;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Inpatient Silver
CREATE OR REPLACE TABLE silver.inp_claims AS
SELECT 
    sha2(concat(DESYNPUF_ID, 'HEALTHCARE_SALT_2026'), 256) AS patient_id_hashed,
    CLM_ID AS claim_id,
    (CAST(CLM_PMT_AMT AS DOUBLE) + CAST(NCH_BENE_IP_DDCTBL_AMT AS DOUBLE)) AS total_cost,
    ICD9_DGNS_CD_1 AS primary_diagnosis,
    'Inpatient' AS claim_type
FROM bronze.inp_claims;

-- Outpatient Silver
CREATE OR REPLACE TABLE silver.out_claims AS
SELECT 
    sha2(concat(DESYNPUF_ID, 'HEALTHCARE_SALT_2026'), 256) AS patient_id_hashed,
    CLM_ID AS claim_id,
    (CAST(CLM_PMT_AMT AS DOUBLE) + CAST(NCH_BENE_PTB_DDCTBL_AMT AS DOUBLE)) AS total_cost,
    'Outpatient' AS claim_type
FROM bronze.out_claims;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Carrier Silver (Doctor visits)
CREATE OR REPLACE TABLE silver.car_claims AS
SELECT 
    sha2(concat(DESYNPUF_ID, 'HEALTHCARE_SALT_2026'), 256) AS patient_id_hashed,
    CLM_ID AS claim_id,
    sha2(concat(TAX_NUM_1, 'PROV_SALT'), 256) AS provider_id_hashed,
    (CAST(LINE_NCH_PMT_AMT_1 AS DOUBLE) + CAST(LINE_BENE_PTB_DDCTBL_AMT_1 AS DOUBLE)) AS total_cost,
    'Carrier' AS claim_type
FROM bronze.car_claims;

-- Pharmacy Silver (PDE)
CREATE OR REPLACE TABLE silver.prescription_claims AS
SELECT 
    sha2(concat(DESYNPUF_ID, 'HEALTHCARE_SALT_2026'), 256) AS patient_id_hashed,
    PDE_ID AS claim_id,
    CAST(TOT_RX_CST_AMT AS DOUBLE) AS total_cost,
    'Pharmacy' AS claim_type
FROM bronze.pde_claims;

num_affected_rows,num_inserted_rows


In [0]:
%sql
show tables from silver

database,tableName,isTemporary
silver,ben_sum,false
silver,car_claims,false
silver,inp_claims,false
silver,outp_claims,false
silver,prescription_claims,false


## Gold Layer Configuration

In [0]:
%sql
create schema if not exists gold 

In [0]:
%sql
CREATE OR REPLACE TABLE gold.patient_risk_360 AS
WITH inpatient_agg AS (
    SELECT patient_id_hashed, SUM(total_cost) as ip_spend FROM silver.inp_claims GROUP BY 1
),
outpatient_agg AS (
    SELECT patient_id_hashed, SUM(total_cost) as op_spend FROM silver.out_claims GROUP BY 1
),
carrier_agg AS (
    SELECT 
        patient_id_hashed, 
        SUM(total_cost) as car_spend,
        COUNT(DISTINCT provider_id_hashed) as unique_providers 
    FROM silver.car_claims GROUP BY 1
),
pharmacy_agg AS (
    SELECT patient_id_hashed, SUM(total_cost) as rx_spend FROM silver.prescription_claims GROUP BY 1
)
SELECT 
    b.patient_id_hashed,
    b.gender,
    b.state_code,
    -- 1. IDENTIFY THE HIGH-RISK COHORT
    b.flag_diabetes,
    b.flag_kidney_disease,
    CASE WHEN b.flag_diabetes = 1 AND b.flag_kidney_disease = 1 THEN 1 ELSE 0 END as is_diabetic_ckd_high_risk,
    
    -- 2. AGGREGATE FINANCIALS (Handling Nulls as 0)
    COALESCE(i.ip_spend, 0) as inpatient_spend,
    COALESCE(o.op_spend, 0) as outpatient_spend,
    COALESCE(c.car_spend, 0) as carrier_spend,
    COALESCE(p.rx_spend, 0) as pharmacy_spend,
    (COALESCE(i.ip_spend, 0) + COALESCE(o.op_spend, 0) + COALESCE(c.car_spend, 0) + COALESCE(p.rx_spend, 0)) as total_cost_of_care,
    
    -- 3. FRAGMENTATION & UTILIZATION INSIGHTS
    COALESCE(c.unique_providers, 0) as provider_count,
    CASE 
        WHEN (COALESCE(c.car_spend, 0) + COALESCE(p.rx_spend, 0)) = 0 THEN 999 -- High ratio indicates "Rescue Care" vs "Preventive"
        ELSE (COALESCE(i.ip_spend, 0) / (COALESCE(c.car_spend, 0) + COALESCE(p.rx_spend, 0))) 
    END as rescue_to_preventive_ratio

FROM silver.ben_sum b
LEFT JOIN inpatient_agg i ON b.patient_id_hashed = i.patient_id_hashed
LEFT JOIN outpatient_agg o ON b.patient_id_hashed = o.patient_id_hashed
LEFT JOIN carrier_agg c ON b.patient_id_hashed = c.patient_id_hashed
LEFT JOIN pharmacy_agg p ON b.patient_id_hashed = p.patient_id_hashed;

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from gold.patient_risk_360 limit 100

patient_id_hashed,gender,state_code,flag_diabetes,flag_kidney_disease,is_diabetic_ckd_high_risk,inpatient_spend,outpatient_spend,carrier_spend,pharmacy_spend,total_cost_of_care,provider_count,rescue_to_preventive_ratio
c84b54b3abc1434b50c915c5fc16e8c2edb5a8a4308d03ee74474c6824b3764d,Male,26,0,0,0,5100.0,50.0,240.0,14360.0,19750.0,5,0.3493150684931507
d168e86d67b99a79fb76b6d7bdd399f5f4d2803b92c6f463f233da37d3cea1b4,Male,39,1,1,1,56304.0,60.0,2230.0,2080.0,60674.0,32,13.063573085846867
05ab2909ef23c15c905725791656a80aeacc8e491ae6379e77e2f75fe307dcad,Female,39,0,0,0,0.0,30.0,10.0,3000.0,3040.0,1,0.0
dbcf6c8fe569e3470ea94e76186be84edf7c96717c612acca0472b1c80ca52b2,Male,6,0,0,0,0.0,0.0,80.0,670.0,750.0,3,0.0
8907f57a543719ea052131213c78478a3701bfe5daae4e096cac706876f5f16c,Male,52,0,0,0,0.0,160.0,1230.0,220.0,1610.0,8,0.0
32169429534c8fa8d7795062e7070fc4b007bb8ea81c907e32ef8b47caf82f60,Male,33,0,1,0,0.0,0.0,570.0,490.0,1060.0,12,0.0
43af79013efe8c666ee795c7833b193b5b09185c7bcc68985b1241109f53127b,Male,39,1,1,1,0.0,2920.0,8630.0,110.0,11660.0,73,0.0
72919306e1a832aa30b5b9e9fc2223092dce55c5ae54e2cb34bc7573eed050e0,Male,24,0,0,0,0.0,2190.0,2230.0,10.0,4430.0,17,0.0
31a7a8d09cbd0f268a460ac9c9150c44b6daf9fdc79dd5ba14f9b07a58b5af42,Female,23,1,0,0,0.0,590.0,1430.0,60.0,2080.0,16,0.0
8f30e79c7895055fe797a27394d251dc0cd47a4cf5debc6701a90d004e422de0,Female,1,0,0,0,0.0,0.0,370.0,2850.0,3220.0,3,0.0


In [0]:
%sql
SELECT 
    is_diabetic_ckd_high_risk,
    COUNT(*) AS patient_count,
    ROUND(AVG(total_cost_of_care), 2) AS avg_total_spend,
    ROUND(AVG(pharmacy_spend), 2) AS avg_pharmacy_spend,
    ROUND(AVG(provider_count), 1) AS avg_unique_doctors,
    ROUND(AVG(rescue_to_preventive_ratio), 2) AS avg_risk_ratio
FROM gold.patient_risk_360
GROUP BY 1;

is_diabetic_ckd_high_risk,patient_count,avg_total_spend,avg_pharmacy_spend,avg_unique_doctors,avg_risk_ratio
0,105166,11222.24,2765.91,24.2,119.1
1,11186,32166.2,4475.56,51.7,5.18
